In [19]:
import json
import numpy as np
from scipy.special import expit
from torchvision import datasets

# Activation functions
activation_functions = {
    "relu": lambda x: np.maximum(0, x),
    "sigmoid": expit,
    "tanh": np.tanh,
    "linear": lambda x: x,
    "softmax": lambda x: np.exp(x - np.max(x)) / np.sum(np.exp(x - np.max(x)), keepdims=True)  # Stable softmax
}

# Load the JSON data
with open("node_based_model.json", "r") as f:
    data = json.load(f)

# Store computed neuron outputs
outputs = {}
layer_outputs = {}  # Store layer-wise outputs to pass as vectors

def process_layer(layer_name, input_vector):
    """
    Process an entire layer at once, passing aggregated outputs to the next layer.
    """
    layer = data[layer_name]
    next_layer_inputs = {}  # Collect outputs to send to the next layer
    
    for neuron in layer["nodes"]:
        neuron_id = neuron["id"]
        weights = np.array(neuron.get("weights", []), dtype=np.float32)
        bias = np.array(neuron.get("biases", 0.0), dtype=np.float32)
        activation = activation_functions.get(neuron.get("activation", "linear"), lambda x: x)
        next_nodes = neuron.get("next_nodes", [])
        
        if weights.shape[0] != input_vector.shape[0]:
            print(f"ERROR: Shape mismatch in Neuron {neuron_id}. Weights: {weights.shape}, Inputs: {input_vector.shape}")
            continue
        
        output = activation(np.dot(weights, input_vector) + bias)
        outputs[neuron_id] = output
        
        # Collect outputs for the next layer
        for next_node in next_nodes:
            if next_node not in next_layer_inputs:
                next_layer_inputs[next_node] = []
            next_layer_inputs[next_node].append(output)
    
    return next_layer_inputs

def forward_pass(input_vector):
    """Process the entire network from input to output."""
    outputs.clear()  # Reset outputs for each new forward pass
    print(f"Initial input vector shape: {input_vector.shape}")
    
    # Process each layer sequentially
    layer_names = list(data.keys())
    layer_input = input_vector
    
    for layer_name in layer_names:
        next_layer_inputs = process_layer(layer_name, layer_input)
        
        if next_layer_inputs:
            layer_input = np.array([np.mean(vals) for vals in next_layer_inputs.values()], dtype=np.float32)
        else:
            break  # Stop if no next layer inputs are found

def validate_output_neurons(output_layer_neurons):
    """Ensure all expected output neurons exist in computed outputs."""
    missing_neurons = [n for n in output_layer_neurons if n not in outputs]
    if missing_neurons:
        print(f"WARNING: The following output neurons were not computed: {missing_neurons}")
    return np.array([outputs.get(neuron_id, 0.0) for neuron_id in output_layer_neurons], dtype=np.float32)

# Load MNIST dataset (without PyTorch DataLoader)
mnist_dataset = datasets.MNIST(root="./data", train=False, download=True)

def preprocess_image(index):
    """Load and preprocess an MNIST image with the correct normalization"""
    image, label = mnist_dataset[index]
    image = np.array(image, dtype=np.float32) / 255.0  # Normalize to [0,1]
    image = (image - 0.5) / 0.5  # Normalize to [-1,1]
    image = image.flatten()  # Flatten the image to a vector
    return image, label

# Test the network on an MNIST sample
index = 0  # Change this to test different images
input_vector, true_label = preprocess_image(index)
forward_pass(input_vector)

# Identify the output neurons (assuming last layer contains exactly 10 neurons for digits 0-9)
output_layer_neurons = [neuron["id"] for neuron in data[list(data.keys())[-1]]["nodes"]]
output_values = validate_output_neurons(output_layer_neurons)

# Apply softmax to output layer if necessary
if "softmax" in [neuron["activation"] for neuron in data[list(data.keys())[-1]]["nodes"]]:
    output_values = activation_functions["softmax"](output_values)

predicted_label = np.argmax(output_values)  # Find index of max value

# Print the output neuron activations and prediction
print("Final Output Neurons:", output_values)
print("Predicted Label:", predicted_label)
print("True Label:", true_label)


Initial input vector shape: (784,)
ERROR: Shape mismatch in Neuron v0_10. Weights: (128,), Inputs: (10,)
ERROR: Shape mismatch in Neuron v0_11. Weights: (128,), Inputs: (10,)
ERROR: Shape mismatch in Neuron v0_12. Weights: (128,), Inputs: (10,)
ERROR: Shape mismatch in Neuron v0_13. Weights: (128,), Inputs: (10,)
ERROR: Shape mismatch in Neuron v0_14. Weights: (128,), Inputs: (10,)
ERROR: Shape mismatch in Neuron v0_15. Weights: (128,), Inputs: (10,)
ERROR: Shape mismatch in Neuron v0_16. Weights: (128,), Inputs: (10,)
ERROR: Shape mismatch in Neuron v0_17. Weights: (128,), Inputs: (10,)
ERROR: Shape mismatch in Neuron v0_18. Weights: (128,), Inputs: (10,)
ERROR: Shape mismatch in Neuron v0_19. Weights: (128,), Inputs: (10,)
Final Output Neurons: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
Predicted Label: 0
True Label: 7


In [25]:
import json
import numpy as np
from scipy.special import expit  # Sigmoid function
from torchvision import datasets, transforms

# Load model from JSON file
def load_model(json_path):
    with open(json_path, 'r') as f:
        return json.load(f)

# Define activation functions
activation_functions = {
    "relu": lambda x: np.maximum(0, x),
    "sigmoid": expit,
    "tanh": np.tanh,
    "linear": lambda x: x,
    "softmax": lambda x: np.exp(x) / np.sum(np.exp(x), axis=0, keepdims=True),
}

# Neural Network Class - Layer-Based
class LayerBasedNN:
    def __init__(self, model_json):
        self.layers = []
        self.activations = []

        for layer_key in sorted(model_json.keys(), key=lambda x: int(x.split('_')[1])):
            layer = model_json[layer_key]
            self.layers.append((np.array(layer["weights"]).T, np.array(layer["biases"])))  # Transpose weights
            self.activations.append(activation_functions[layer["activation"]])
    
    def layer_function(self, x, weights, biases, activation_fn):
        """Processes a single layer."""
        return activation_fn(np.dot(x, weights) + biases)
    
    def forward(self, x):
        """Executes forward pass layer by layer."""
        for (weights, biases), activation in zip(self.layers, self.activations):
            x = self.layer_function(x, weights, biases, activation)
        return x

# Load model from JSON
model_path = "model.json"
model_json = load_model(model_path)
nn = LayerBasedNN(model_json)

# Load MNIST data
mnist_test = datasets.MNIST(root="./data", train=False, transform=transforms.ToTensor(), download=True)

# Test model on an MNIST image
image, label = mnist_test[0]
image = image.numpy().reshape(-1)  # Flatten image
prediction = nn.forward(image)
predicted_label = np.argmax(prediction)

print(f"Predicted label: {predicted_label}, Actual label: {label}")

Predicted label: 7, Actual label: 7


In [37]:
import json
import numpy as np
from scipy.special import expit  # Sigmoid function
from torchvision import datasets, transforms

# Load the JSON model
def load_model(json_path):
    with open(json_path, 'r') as f:
        model = json.load(f)
    return model

# Define activation functions
activation_functions = {
    "relu": lambda x: np.maximum(0, x),
    "sigmoid": expit,
    "tanh": np.tanh,
    "linear": lambda x: x,
    "softmax": lambda x: np.exp(x) / np.sum(np.exp(x), axis=0, keepdims=True)  # Softmax for 1D input
}

# Reverse engineer prev_nodes mapping from next_nodes
def build_prev_nodes_mapping(model_json):
    prev_nodes_map = {}
    for layer in model_json.values():
        for neuron in layer["nodes"]:
            prev_nodes_map[neuron["id"]] = []
    
    for layer in model_json.values():
        for neuron in layer["nodes"]:
            for next_node in neuron["next_nodes"]:
                prev_nodes_map[next_node].append(neuron["id"])

    return prev_nodes_map

# Fire each neuron individually when inputs are ready
def process_neuron(neuron, neuron_outputs):
    prev_nodes = prev_nodes_map[neuron["id"]]
    
    # Ensure all previous nodes have fired before calculating this neuron
    if not all(node in neuron_outputs for node in prev_nodes):
        return None  # Wait until all dependencies are resolved

    # Gather inputs from previous neurons
    inputs = np.array([neuron_outputs[node] for node in prev_nodes]) if prev_nodes else neuron_outputs["input"]
    
    weights = np.array(neuron["weights"])
    bias = neuron["biases"]
    activation = activation_functions[neuron["activation"]]

    # Ensure correct input size
    if len(inputs) != len(weights):
        raise ValueError(f"Input size mismatch in Neuron {neuron['id']}. Expected {len(weights)} values, got {len(inputs)}")

    weighted_sum = np.dot(inputs, weights) + bias
    output = activation(weighted_sum)

    print(f"Neuron {neuron['id']}: Weighted Sum = {weighted_sum}, Output = {output}")
    return output

# Process neurons layer by layer, but firing each neuron as soon as its inputs are available
def process_layer(layer, neuron_outputs):
    remaining_neurons = list(layer["nodes"])

    while remaining_neurons:
        next_batch = []
        for neuron in remaining_neurons:
            output = process_neuron(neuron, neuron_outputs)
            if output is not None:
                neuron_outputs[neuron["id"]] = output
            else:
                next_batch.append(neuron)  # Keep neurons that still need inputs

        if len(next_batch) == len(remaining_neurons):  # No progress -> circular dependency
            raise RuntimeError(f"Circular dependency detected in layer: {layer['id']}")

        remaining_neurons = next_batch  # Update remaining neurons

# Load the model from JSON
model_path = "node_based_model.json"
model_json = load_model(model_path)

# Build the prev_nodes mapping
prev_nodes_map = build_prev_nodes_mapping(model_json)

# Load MNIST dataset
mnist_test = datasets.MNIST(root="./data", train=False, transform=transforms.ToTensor(), download=True)

# Test on a single image
image, label = mnist_test[0]
image = image.numpy().reshape(-1)  # Flatten 28x28 image into a vector

# Initialize neuron outputs
neuron_outputs = {"input": image}

# Process layers sequentially, ensuring neurons fire when ready
for layer_key in sorted(model_json.keys(), key=lambda x: int(x.split('_')[1])):
    layer = model_json[layer_key]
    process_layer(layer, neuron_outputs)

# Extract final output neurons
final_outputs = np.array([neuron_outputs[node["id"]] for node in model_json[layer_key]["nodes"]])
predicted_label = np.argmax(final_outputs)

print(f"Final Output Neurons: {final_outputs}")
print(f"Predicted Label: {predicted_label}, Actual Label: {label}")


Neuron v0_00: Weighted Sum = -2.4674497316817074, Output = 0.0
Neuron v0_01: Weighted Sum = 1.2614272282631034, Output = 1.2614272282631034
Neuron v0_02: Weighted Sum = -0.23895092706418275, Output = 0.0
Neuron v0_03: Weighted Sum = -1.4914002860764435, Output = 0.0
Neuron v0_04: Weighted Sum = 3.117748057827733, Output = 3.117748057827733
Neuron v0_05: Weighted Sum = 0.31698690164095567, Output = 0.31698690164095567
Neuron v0_06: Weighted Sum = 3.1259155516969326, Output = 3.1259155516969326
Neuron v0_07: Weighted Sum = -0.07143255928815356, Output = 0.0
Neuron v0_08: Weighted Sum = 2.819468526551663, Output = 2.819468526551663
Neuron v0_09: Weighted Sum = 0.22640140615198281, Output = 0.22640140615198281
Neuron v0_010: Weighted Sum = -0.008666960421510342, Output = 0.0
Neuron v0_011: Weighted Sum = -0.11886079330957217, Output = 0.0
Neuron v0_012: Weighted Sum = 2.5216352654524785, Output = 2.5216352654524785
Neuron v0_013: Weighted Sum = 0.05993900340864848, Output = 0.0599390034086